#Lluvia
dataset de clsificacion para predecir la si el dia siguite lloverá

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import torch

%matplotlib inline

from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial 1/Laboratorios/ExamenLab1/weatherAUS.csv")
df.head(10)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
5,2008-12-06,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,No
6,2008-12-07,Albury,14.3,25.0,0.0,NaN,NaN,W,50.0,SW,...,49.0,19.0,1009.6,1008.2,1.0,NaN,18.1,24.6,No,No
7,2008-12-08,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,No
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
9,2008-12-10,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,No


In [ ]:
# Reemplazar valores 'NA' con NaN
df.replace('NA', np.nan, inplace=True)
# Convertir 'Date' a tipo datetime
df['Date'] = pd.to_datetime(df['Date'])
# Convertir valores numéricos a tipo float
numeric_cols = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
                'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
                'Temp9am', 'Temp3pm']
df[numeric_cols] = df[numeric_cols].astype(float)
# Eliminar columnas no deseadas
df.drop(['Evaporation', 'Sunshine'], axis=1, inplace=True)
# Renombrar columnas
df.rename(columns={'WindDir9am': 'WindDirection9am', 'WindDir3pm': 'WindDirection3pm'}, inplace=True)
# Convertir 'RainToday' a variables dummy
df = pd.get_dummies(df, columns=['RainToday'], drop_first=True)

In [ ]:
# Convertir las columnas categóricas en variables dummy (one-hot encoding)
categorical_cols = ['Location', 'WindGustDir', 'WindDirection9am', 'WindDirection3pm']
df = pd.get_dummies(df, columns=categorical_cols)

In [ ]:
# Identificar columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Reemplazar valores faltantes en columnas numéricas con la media
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
# Reemplazar valores faltantes en columnas categóricas con una categoría "desconocida"
df.fillna('Desconocido', inplace=True)

In [ ]:
# Convertir todas las columnas a tipo numérico
df = df.apply(pd.to_numeric, errors='ignore')
# Verificar los tipos de datos después de la conversión
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Columns: 114 entries, Date to WindDirection3pm_WSW
dtypes: bool(98), float64(14), int64(1), object(1)
memory usage: 31.4+ MB


In [ ]:
print(df.shape)

(145460, 114)


In [ ]:
# Calcular la cantidad de valores diferentes en una co
valores = df['RainTomorrow'].unique()
print(valores)

['No' 'Yes' 'Desconocido']


In [ ]:
# Reemplazar valores en la columna 'B'
df['RainTomorrow'].replace({'Desconocido':'Yes'}, inplace=True)

In [ ]:
#variables categoricas
columnas_categoricas = df.select_dtypes(include=['bool','object']).columns

In [ ]:
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  df[columna] = le.fit_transform(df[columna])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Columns: 114 entries, Date to WindDirection3pm_WSW
dtypes: float64(14), int64(100)
memory usage: 126.5 MB


Separamos los datos- Caracteristicas "**X**" y  Variable objetivo "**y**" para usarlo mas adelante

In [ ]:
# Extrae la columna 'RainTomorrow' y la asigna a la variable y
y = df['RainTomorrow'].values
# Elimina la columna 'RainTomorrow' de la matriz X
X = df.drop(columns=['RainTomorrow']).values

In [ ]:
print(X)

[[1.22808960e+18 1.34000000e+01 2.29000000e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.22817600e+18 7.40000000e+00 2.51000000e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.22826240e+18 1.29000000e+01 2.57000000e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 ...
 [1.49817600e+18 5.40000000e+00 2.69000000e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.49826240e+18 7.80000000e+00 2.70000000e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.49834880e+18 1.49000000e+01 2.32213483e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [ ]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
X = np.concatenate([np.ones((m, 1)), X], axis=1)

nota: siempre normalizar

In [ ]:
from sklearn.preprocessing import StandardScaler
# Crea un objeto StandardScaler
scaler = StandardScaler()
# Ajusta y transforma toda la matriz de características
X_norm = scaler.fit_transform(X)

In [ ]:
# Revisamos
X_norm

array([[ 0.        , -1.72187528,  0.18944661, ..., -0.27330429,
         3.92322649, -0.26460377],
       [ 0.        , -1.72078953, -0.75310073, ..., -0.27330429,
        -0.25489224,  3.77923569],
       [ 0.        , -1.71970377,  0.110901  , ..., -0.27330429,
        -0.25489224,  3.77923569],
       ...,
       [ 0.        ,  1.67218871, -1.06728318, ..., -0.27330429,
         3.92322649, -0.26460377],
       [ 0.        ,  1.67327446, -0.69026424, ..., -0.27330429,
        -0.25489224, -0.26460377],
       [ 0.        ,  1.67436022,  0.42508345, ..., -0.27330429,
        -0.25489224, -0.26460377]])

In [ ]:
X.shape, y.shape

((145460, 114), (145460,))

In [ ]:
# funciones de pérdida
def bce(y, y_hat):
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return - np.mean(y*np.log(y_hat) - (1 - y)*np.log(1 - y_hat))

# funciones de activación
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# Perceptron
class Perceptron():
  def __init__(self, size, activation, loss):
    self.w = np.random.randn(size)
    self.ws = []
    self.activation = activation
    self.loss = loss

  def __call__(self, w, x):
    return self.activation(np.dot(x, w))

  def fit(self, x, y, epochs, lr):
    x = np.c_[np.ones(len(x)), x]
    for epoch in range(epochs):
        # Batch Gradient Descent
        y_hat = self(self.w, x)
        # función de pérdida
        l = self.loss(y, y_hat)
        # derivadas
        dldh = (y_hat - y)
        dhdw = x
        dldw = np.dot(dldh, dhdw)
        # actualizar pesos
        self.w = self.w - lr*dldw
        self.ws.append(self.w.copy())

In [ ]:
from sklearn.model_selection import train_test_split
# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

In [ ]:
perceptron = Perceptron(115, sigmoid, bce)
epochs, lr = 50, 0.01
perceptron.fit(X_train, y_train, epochs, lr)

pesos_finales = perceptron.w
print(pesos_finales)

<ipython-input-19-33217cc90b54>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


[-611.32920062   -1.19649297   14.9531197    72.08998721  -25.49124612
   29.19194035  208.56854464  -17.33666648  -42.32092488    4.68940173
  340.97110606   82.49184699 -254.71644043   40.5219938   101.07030842
   88.39554948   37.83324786   -1.60093234   53.48142813  -22.61266943
    4.30374683  -47.57457892   28.53276489  -47.66842208  -12.82346387
   37.67155507  -13.71290299  -10.41158474    1.66259637   13.75839711
    3.73854571  -50.06320259    0.78032191  -30.28595241  -39.42433105
  -29.61710467   64.17009798  -30.84878165   -8.08859451   18.66355583
   -9.91133515  -68.63510765   -7.04925027  -13.14094315    2.94547275
    3.43410323   -5.69850414   43.72507222   44.11565241   36.66928248
   23.69141277  -16.71659725   22.2224167   -25.1639557    32.97190671
   12.26692953    4.67177098  -58.7717978     7.05935154   -7.60574397
   12.13450294   23.95633785  -13.07843241   59.70239128   33.46925482
  -21.65256253  -22.7990582   -54.66960767  -44.84862885  -35.37219485
  -55.

In [ ]:
def presicion(x, weights):
    x = np.c_[np.ones(len(x)), x]
    return sigmoid(np.dot(x, weights))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

y_train_pred = np.round(presicion(X_train, perceptron.w))
y_test_pred = np.round(presicion(X_test, perceptron.w))

# Calcular la precisión para el conjunto de entrenamiento y prueba
train_accuracy = accuracy_score(y_train, y_train_pred)*100
test_accuracy = accuracy_score(y_test, y_test_pred)*100

print(f"Precisión en conjunto de entrenamiento: {train_accuracy:.2f} %")
print(f"Precisión en conjunto de prueba: {test_accuracy:.2f} %")

Precisión en conjunto de entrenamiento: 81.01
Precisión en conjunto de prueba: 80.90


<ipython-input-19-33217cc90b54>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
